In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web


The following command must be run outside of the IPython shell:

    $ pip install python-zillow

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [ ]:
api.

In [29]:
from zillow import ValuationApi

zid = 'X1-ZWz18cqbc8wgsr_7j3qs'

api = zillow.ValuationApi()

#zpid = "2100641621"



data = api.GetSearchResults(zid,address,postal_code)

data.zestiamte.amount



address = "3408 N Venice ST., Arlington, VA"
postal_code = "22207"

rochester = pd.read_csv('./us/ny/city_of_rochester.csv')

rochester.drop(['DISTRICT','REGION','ID','HASH'], axis = 1, inplace = True)

In [108]:
rochester

,LON,LAT,NUMBER,STREET,UNIT,CITY,POSTCODE
0,-77.634017,43.188276,124,ALBEMARLE ST,NaN,ROCHESTER,14613.0
1,-77.635219,43.188275,164,ALBEMARLE ST,NaN,ROCHESTER,14613.0
2,-77.635817,43.188274,188,ALBEMARLE ST,NaN,ROCHESTER,14613.0
3,-77.637223,43.188274,240,ALBEMARLE ST,NaN,ROCHESTER,14613.0
4,-77.637556,43.188446,357,SENECA PKWY,NaN,ROCHESTER,14613.0
5,-77.637457,43.186840,235,ALAMEDA ST,NaN,ROCHESTER,14613.0
6,-77.631179,43.186745,1117-1119,LAKE AV,NaN,ROCHESTER,14613.0
7,-77.638447,43.186598,1126,DEWEY AV,NaN,ROCHESTER,14613.0
8,-77.632826,43.186519,68,AUGUSTINE ST,NaN,ROCHESTER,14613.0
9,-77.633950,43.186481,110-112,AUGUSTINE ST,NaN,ROCHESTER,14613.0


In [139]:
rochester = rochester.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [140]:
address = str(rochester['NUMBER']+' '+rochester['STREET']+'.,'+rochester['CITY']+', NY')
postal_code = str(rochester['POSTCODE'])

In [143]:
print(address)

0            124 ALBEMARLE ST.,ROCHESTER, NY
1            164 ALBEMARLE ST.,ROCHESTER, NY
2            188 ALBEMARLE ST.,ROCHESTER, NY
3            240 ALBEMARLE ST.,ROCHESTER, NY
4             357 SENECA PKWY.,ROCHESTER, NY
5              235 ALAMEDA ST.,ROCHESTER, NY
6           1117-1119 LAKE AV.,ROCHESTER, NY
7               1126 DEWEY AV.,ROCHESTER, NY
8             68 AUGUSTINE ST.,ROCHESTER, NY
9        110-112 AUGUSTINE ST.,ROCHESTER, NY
10           234 AUGUSTINE ST.,ROCHESTER, NY
11           378 AUGUSTINE ST.,ROCHESTER, NY
12           458 AUGUSTINE ST.,ROCHESTER, NY
13           486 AUGUSTINE ST.,ROCHESTER, NY
14           492 AUGUSTINE ST.,ROCHESTER, NY
15       518-520 AUGUSTINE ST.,ROCHESTER, NY
16           552 AUGUSTINE ST.,ROCHESTER, NY
17           568 AUGUSTINE ST.,ROCHESTER, NY
18               1088 LAKE AV.,ROCHESTER, NY
19               1080 LAKE AV.,ROCHESTER, NY
20               1070 LAKE AV.,ROCHESTER, NY
21                 71 BIRR ST.,ROCHESTER, NY
22        

In [147]:
address = '124 ALBEMARLE ST.,ROCHESTER, NY'
postal_code = '14613'

In [148]:
data = api.GetSearchResults(zid,address,postal_code)

data.zestiamte.amount

86896

ZillowError: {'message': 'Zillow did not return a valid response: <?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/18f7df5/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>131 Fish Hook Rd., Oxford, ME</address><citystatezip>04270</citystatezip></request><message><text>Request successfully processed</text><code>0</code></message><response><results><result><zpid>224630407</zpid><links><homedetails>https://www.zillow.com/homedetails/131-Fish-Hook-Rd-Oxford-ME-04270/224630407_zpid/</homedetails><graphsanddata>http://www.zillow.com/homedetails/131-Fish-Hook-Rd-Oxford-ME-04270/224630407_zpid/#charts-and-data</graphsanddata><mapthishome>http://www.zillow.com/homes/224630407_zpid/</mapthishome><comparables>http://www.zillow.com/homes/comps/22463